In [1]:
import os
os.getcwd()
os.chdir("H:\pyspark_advanced-coding_interview")
print(os.getcwd())


from pyspark.sql import SparkSession

# Create a Spark session with optimized settings
spark = (
    SparkSession.builder 
    .appName("OptimizedLocalSpark") 
    .config("spark.driver.memory", "8g")        
    .config("spark.executor.memory", "8g")    
    .config("spark.executor.cores", "4")       
    .config("spark.cores.max", "12")           
    .config("spark.sql.shuffle.partitions", "28")  
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") 
    .getOrCreate()
)
sc = spark.sparkContext

H:\pyspark_advanced-coding_interview


In [2]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("EmployeeQueries").getOrCreate()

# Sample data for employees
data = [
    (1, 'Alice', 'Engineering', 80000, None),     # Manager of Engineering
    (2, 'Bob', 'Engineering', 70000, 1),
    (3, 'Charlie', 'Engineering', 60000, 1),
    (4, 'David', 'Marketing', 90000, None),       # Manager of Marketing
    (5, 'Eve', 'Marketing', 55000, 4),
    (6, 'Frank', 'Marketing', 65000, 4),
    (7, 'Grace', 'Sales', 75000, None),           # Manager of Sales
    (8, 'Hannah', 'Sales', 70000, 7),
    (9, 'Ian', 'Sales', 65000, 7)
]

columns = ["employee_id", "name", "department", "salary", "manager_id"]

# Create DataFrame
df = spark.createDataFrame(data, columns)

# Create temporary table for SQL queries
df.createOrReplaceTempView("employee_table")
df.show()


+-----------+-------+-----------+------+----------+
|employee_id|   name| department|salary|manager_id|
+-----------+-------+-----------+------+----------+
|          1|  Alice|Engineering| 80000|      null|
|          2|    Bob|Engineering| 70000|         1|
|          3|Charlie|Engineering| 60000|         1|
|          4|  David|  Marketing| 90000|      null|
|          5|    Eve|  Marketing| 55000|         4|
|          6|  Frank|  Marketing| 65000|         4|
|          7|  Grace|      Sales| 75000|      null|
|          8| Hannah|      Sales| 70000|         7|
|          9|    Ian|      Sales| 65000|         7|
+-----------+-------+-----------+------+----------+



# Employees with highest salary in a department

In [7]:
res = spark.sql(""" 
select Employee_id, salary, dense_rnk 
from (
select *,
dense_rank() over ( partition by department order by salary desc)  as dense_rnk  
from  employee_table         
) A
where dense_rnk = 1 ;             
                """)
res.show()

+-----------+------+---------+
|Employee_id|salary|dense_rnk|
+-----------+------+---------+
|          1| 80000|        1|
|          4| 90000|        1|
|          7| 75000|        1|
+-----------+------+---------+



In [8]:
res1 = spark.sql(""" 
select department, max(salary) as dept_max_salary  from employee_table 
group by department 
order by max(salary)
          
                """)
res1.show()

+-----------+---------------+
| department|dept_max_salary|
+-----------+---------------+
|      Sales|          75000|
|Engineering|          80000|
|  Marketing|          90000|
+-----------+---------------+



In [11]:
res2 = spark.sql(""" 
select department, name, salary 
from employee_table e
where salary = ( Select max(salary) from employee_table where department = e.department)
order by salary

          
                """)
res2.show()

+-----------+-----+------+
| department| name|salary|
+-----------+-----+------+
|      Sales|Grace| 75000|
|Engineering|Alice| 80000|
|  Marketing|David| 90000|
+-----------+-----+------+



# Employees with Salary lesser than department average

In [12]:
res3= spark.sql(""" 
select department, name, salary 
from employee_table e
where salary < ( Select avg(salary) from employee_table where department = e.department)
order by salary

          
                """)
res3.show()

+-----------+-------+------+
| department|   name|salary|
+-----------+-------+------+
|  Marketing|    Eve| 55000|
|Engineering|Charlie| 60000|
|  Marketing|  Frank| 65000|
|      Sales|    Ian| 65000|
+-----------+-------+------+



In [13]:
res4= spark.sql(""" 
 with dpt_avg as(              
select department, avg(salary) as dpt_avg_salary
from employee_table
group by department 
)


select e.department, e.name, e.salary, d.dpt_avg_salary
from employee_table e
inner join dpt_avg d
on e.department = d.department 
where e.salary < d.dpt_avg_salary


          
                """)
res4.show()


+-----------+-------+------+--------------+
| department|   name|salary|dpt_avg_salary|
+-----------+-------+------+--------------+
|Engineering|Charlie| 60000|       70000.0|
|  Marketing|    Eve| 55000|       70000.0|
|  Marketing|  Frank| 65000|       70000.0|
|      Sales|    Ian| 65000|       70000.0|
+-----------+-------+------+--------------+



In [14]:
res5= spark.sql(""" 
SELECT e.department, e.name, e.salary
FROM employee_table e
JOIN (
    SELECT department, AVG(salary) AS avg_salary
    FROM employee_table
    GROUP BY department
) dept_avg
ON e.department = dept_avg.department
WHERE e.salary < dept_avg.avg_salary



          
                """)
res5.show()

+-----------+-------+------+
| department|   name|salary|
+-----------+-------+------+
|Engineering|Charlie| 60000|
|  Marketing|    Eve| 55000|
|  Marketing|  Frank| 65000|
|      Sales|    Ian| 65000|
+-----------+-------+------+



# Employees with Salary Less Than Department Average but More Than Average of Any Other Department

In [16]:
res7= spark.sql(""" 
SELECT e.name, e.salary, e.department
FROM employee_table e
JOIN (
    SELECT department, AVG(salary) AS avg_salary
    FROM employee_table
    GROUP BY department
) dept_avg
ON e.department = dept_avg.department
WHERE e.salary < dept_avg.avg_salary
AND e.salary > (
    SELECT MAX(avg_salary)
    FROM (
        SELECT AVG(salary) AS avg_salary
        FROM employee_table
        GROUP BY department
    ) other_depts
)


          
                """)
res7.show()

+----+------+----------+
|name|salary|department|
+----+------+----------+
+----+------+----------+



In [ ]:
res10= spark.sql(""" 
SELECT e.department, e.name, e.salary
FROM employee_table e
Inner JOIN (
    SELECT department, AVG(salary) AS avg_salary
    FROM employee_table
    GROUP BY department
) dept_avg
ON e.department = dept_avg.department
WHERE e.salary < dept_avg.avg_salary
and e.salary > any(select avg(salary from employee_table groupp by department))



          
                """)
res10.show()

# Employees with the Same Salary

In [6]:
res8= spark.sql(""" 
select e1.employee_id, e1.name as employee1, e1.salary as employee1_salary
FROM employee_table e1
inner join employee_table e2
on e1.salary = e2.salary 
and e1.employee_id <> e2.employee_id 


          
                """)
res8.show()

+-----------+---------+----------------+
|employee_id|employee1|employee1_salary|
+-----------+---------+----------------+
|          6|    Frank|           65000|
|          9|      Ian|           65000|
|          2|      Bob|           70000|
|          8|   Hannah|           70000|
+-----------+---------+----------------+



# Departments Where No Employee Has a Salary Greater Than Their Manager's Salary

In [8]:
df.show()

+-----------+-------+-----------+------+----------+
|employee_id|   name| department|salary|manager_id|
+-----------+-------+-----------+------+----------+
|          1|  Alice|Engineering| 80000|      null|
|          2|    Bob|Engineering| 70000|         1|
|          3|Charlie|Engineering| 60000|         1|
|          4|  David|  Marketing| 90000|      null|
|          5|    Eve|  Marketing| 55000|         4|
|          6|  Frank|  Marketing| 65000|         4|
|          7|  Grace|      Sales| 75000|      null|
|          8| Hannah|      Sales| 70000|         7|
|          9|    Ian|      Sales| 65000|         7|
+-----------+-------+-----------+------+----------+



In [10]:
res9= spark.sql(""" 
select e.employee_id, e.name as employee_name, e.salary as employee_salary, m.name as manager_name
FROM employee_table e
inner join employee_table m
on e.employee_id = m.manager_id 
and e.employee_id > m.employee_id 


          
                """)
res9.show()

+-----------+-------------+---------------+------------+
|employee_id|employee_name|employee_salary|manager_name|
+-----------+-------------+---------------+------------+
+-----------+-------------+---------------+------------+



# Difference between employee salary and average Salary of department

In [14]:
res12 = spark.sql(""" 
SELECT e.department, e.name, e.salary, dept_avg.avg_salary, (e.salary - dept_avg.avg_salary) AS difference
FROM employee_table e
INNER JOIN (
    SELECT department, AVG(salary) AS avg_salary
    FROM employee_table
    GROUP BY department
) dept_avg
ON e.department = dept_avg.department
""")
res12.show()


+-----------+-------+------+----------+----------+
| department|   name|salary|avg_salary|difference|
+-----------+-------+------+----------+----------+
|Engineering|  Alice| 80000|   70000.0|   10000.0|
|Engineering|    Bob| 70000|   70000.0|       0.0|
|Engineering|Charlie| 60000|   70000.0|  -10000.0|
|  Marketing|  David| 90000|   70000.0|   20000.0|
|  Marketing|    Eve| 55000|   70000.0|  -15000.0|
|  Marketing|  Frank| 65000|   70000.0|   -5000.0|
|      Sales|  Grace| 75000|   70000.0|    5000.0|
|      Sales| Hannah| 70000|   70000.0|       0.0|
|      Sales|    Ian| 65000|   70000.0|   -5000.0|
+-----------+-------+------+----------+----------+



In [19]:
res13 = spark.sql(""" 
SELECT employee_id,salary, department, 
avg(salary) over (partition by department) avg_sal_over_dept,
salary - avg(salary) over (partition by department) as diffsal
FROM employee_table 

""")
res13.show()

+-----------+------+-----------+-----------------+--------+
|employee_id|salary| department|avg_sal_over_dept| diffsal|
+-----------+------+-----------+-----------------+--------+
|          1| 80000|Engineering|          70000.0| 10000.0|
|          2| 70000|Engineering|          70000.0|     0.0|
|          3| 60000|Engineering|          70000.0|-10000.0|
|          4| 90000|  Marketing|          70000.0| 20000.0|
|          5| 55000|  Marketing|          70000.0|-15000.0|
|          6| 65000|  Marketing|          70000.0| -5000.0|
|          7| 75000|      Sales|          70000.0|  5000.0|
|          8| 70000|      Sales|          70000.0|     0.0|
|          9| 65000|      Sales|          70000.0| -5000.0|
+-----------+------+-----------+-----------------+--------+



# Employees whose salary is in top 2 percentile in department

In [23]:
res15 = spark.sql(""" 
SELECT employee_id, salary 
FROM 
    (
    SELECT employee_id, salary, department, 
           percent_rank() OVER (PARTITION BY department ORDER BY salary DESC) AS per_rnk 
    FROM employee_table
    ) AS emp
WHERE per_rnk >= 0.98
""")
res15.show()


+-----------+------+
|employee_id|salary|
+-----------+------+
|          3| 60000|
|          5| 55000|
|          9| 65000|
+-----------+------+



# Employees who earn more than every employee in dept no sales

In [26]:
res16 = spark.sql(""" 
SELECT employee_id, salary 
FROM employee_table
WHERE salary > (SELECT MAX(salary) FROM employee_table WHERE department = 'sales')
""")
res16.show()


+-----------+------+
|employee_id|salary|
+-----------+------+
+-----------+------+



# Department names ( with employee name) with more than 2 employee and salary greater than 90% of respective department averge salary

In [29]:
res17 = spark.sql(""" 
SELECT department, 
       COUNT(name) AS dept_employee_count, 
       AVG(salary) AS dept_avg_salary
FROM employee_table
GROUP BY department 
HAVING dept_employee_count > 2 
       AND dept_avg_salary > (0.90 * (SELECT AVG(salary) FROM employee_table WHERE department = department))
ORDER BY dept_avg_salary
""")
res17.show()


+-----------+-------------------+---------------+
| department|dept_employee_count|dept_avg_salary|
+-----------+-------------------+---------------+
|Engineering|                  3|        70000.0|
|  Marketing|                  3|        70000.0|
|      Sales|                  3|        70000.0|
+-----------+-------------------+---------------+



In [36]:
res18 = spark.sql(""" 
SELECT DISTINCT department 
FROM (
    SELECT employee_id, department,
           SUM(CASE WHEN salary > 0.98 * avgsal THEN 1 ELSE 0 END) 
           OVER (PARTITION BY department) AS empCnt
    FROM (
        SELECT employee_id, department, salary, 
               AVG(salary) OVER (PARTITION BY department) AS avgsal 
        FROM employee_table
    ) emp
) emp1 
WHERE empCnt >= 2
""")
res18.show()


+-----------+
| department|
+-----------+
|Engineering|
|      Sales|
+-----------+



# Select top 3 departments with at least 2 employees and rar    them accroding to the percentage fo their employees making over 100K in salary

In [41]:
res19 = spark.sql(""" 
SELECT department,
       100 * SUM(CASE WHEN salary >= 100000 THEN 1 ELSE 0 END) / COUNT(employee_id) AS prcnt
FROM employee_table
GROUP BY department
HAVING COUNT(employee_id) > 2
ORDER BY prcnt DESC
LIMIT 3
""")
res19.show()


+-----------+-----+
| department|prcnt|
+-----------+-----+
|Engineering|  0.0|
|  Marketing|  0.0|
|      Sales|  0.0|
+-----------+-----+

